<a href="https://colab.research.google.com/github/Sciform/sciform-hwz-rl-and-qml/blob/main/quantum_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit_machine_learning
!pip install nlopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.6/231.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.8/436.8 kB 5.8 MB/s eta 0:00:00


In [5]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.optimizers import COBYLA
from qiskit_machine_learning.utils import algorithm_globals

from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.circuit.library import QNNCircuit

algorithm_globals.random_seed = 42

ImportError: cannot import name 'BaseSampler' from 'qiskit.primitives' (/usr/local/lib/python3.11/dist-packages/qiskit/primitives/__init__.py)

In [ ]:
def create_simple_dataset():

  num_inputs = 2
  num_samples = 20
  X = 2 * algorithm_globals.random.random([num_samples, num_inputs]) - 1
  y01 = 1 * (np.sum(X, axis=1) >= 0)  # in { 0,  1}
  y = 2 * y01 - 1  # in {-1, +1}
  y_one_hot = np.zeros((num_samples, 2))
  for i in range(num_samples):
      y_one_hot[i, y01[i]] = 1

  for x, y_target in zip(X, y):
      if y_target == 1:
          plt.plot(x[0], x[1], "bo")
      else:
          plt.plot(x[0], x[1], "go")
  plt.plot([-1, 1], [1, -1], "--", color="black")
  plt.show()

create_simple_dataset()